### 05 — Sessionization & Prefix→Target Generation

**Goal:**  
- For **Amazon Books 2023**: build synthetic sessions using **sliding windows** (Option A) to convert sparse user timelines into many sequence training examples.
- For **MARS**: sessionize by **1 hour gap** (as decided in Step 04).
- For **YOOCHOOSE**: use existing session IDs (already processed).
- Generate `prefix -> target` pairs for training (next-item prediction).
- Save outputs and report diagnostics.

Outputs saved to `data/processed/`:
- `amazon_windows_sessions.parquet` (or chunked files)
- `amazon_prefix_target.parquet` (or chunked)
- `mars_sessions.parquet`
- `mars_prefix_target.parquet`
- summary CSVs


In [2]:
# Cell: imports & settings
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import math
import json

%matplotlib inline

DATA_DIR = Path("../data/processed")
OUT_DIR = DATA_DIR
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Parameters (tune as needed)
WINDOW_SIZE = 20    # number of items in a sliding window
STRIDE = 10         # step between windows
MIN_WINDOW_ITEMS = 2  # min items to keep a window (>=2 gives at least one pair)
MAX_PREFIX_LEN = 50  # cap prefix length when generating prefix-target
MIN_SESSION_LEN = 2
CHUNK_WRITE = True  # if True, write parquet in chunk files to avoid excessive RAM use


#### A — Helper functions
- `build_sliding_windows()` : builds sliding-window sessions for Amazon (memory-efficient).
- `sessionize_by_gap()` : sessionize a dataset by time-gap (used for MARS).
- `generate_prefix_target()` : generate prefix→target pairs from a sessions DataFrame.


In [3]:
# Cell: helper function definitions
def build_sliding_windows(df, user_col='user_id', item_col='item_id',
                          time_col='timestamp', window_size=WINDOW_SIZE, stride=STRIDE,
                          out_session_col='session_id_real'):
    """
    Build sliding-window synthetic sessions from a user timeline DataFrame.
    Returns a DataFrame with columns: dataset, user_id, session_id_real, item_id, timestamp
    This version streams per-user and yields rows to reduce peak memory usage.
    """
    rows = []
    # Ensure sorted
    df = df.sort_values([user_col, time_col])
    users = df[user_col].unique()
    total_users = len(users)
    print(f"Users to process: {total_users:,}")
    
    # iterate per user
    for user in tqdm(users, desc="Building windows per user"):
        user_slice = df[df[user_col] == user]
        items = user_slice[item_col].astype(str).tolist()
        times = pd.to_datetime(user_slice[time_col]).tolist()
        n = len(items)
        if n < MIN_WINDOW_ITEMS:
            continue
        start = 0
        win_idx = 0
        while start < n:
            end = min(start + window_size, n)
            window_items = items[start:end]
            window_times = times[start:end]
            if len(window_items) >= MIN_WINDOW_ITEMS:
                sess_id = f"{user}__w{win_idx}_{start}"
                for it, ts in zip(window_items, window_times):
                    rows.append({
                        "dataset": "amazon_books_2023",
                        "user_id": str(user),
                        out_session_col: sess_id,
                        "item_id": str(it),
                        "timestamp": pd.to_datetime(ts)
                    })
                win_idx += 1
            start += stride
            # if rows accumulate too large, yield in batches (handled by caller)
            if len(rows) >= 500_000:
                yield pd.DataFrame(rows)
                rows = []
    # final flush
    if rows:
        yield pd.DataFrame(rows)


def sessionize_by_gap(df, gap_seconds=3600, user_col='user_id', time_col='timestamp',
                      out_session_col='session_id_real'):
    """
    Sessionize by gap_seconds per user. Returns sessionized df and session summary.
    """
    df = df.sort_values([user_col, time_col]).copy()
    df[time_col] = pd.to_datetime(df[time_col])
    df['prev_ts'] = df.groupby(user_col)[time_col].shift(1)
    df['gap_s'] = (df[time_col] - df['prev_ts']).dt.total_seconds()
    df['new_session'] = (df['gap_s'].isna()) | (df['gap_s'] > gap_seconds)
    df['sess_idx'] = df.groupby(user_col)['new_session'].cumsum().astype(int)
    df[out_session_col] = df[user_col].astype(str) + "__s" + df['sess_idx'].astype(str)
    # summary
    session_summary = df.groupby(out_session_col).agg(
        dataset=('dataset','first'),
        user_id=(user_col,'first'),
        start_time=(time_col,'min'),
        end_time=(time_col,'max'),
        session_length=('item_id','size')
    ).reset_index()
    # filter short and very long sessions
    valid_sessions = session_summary[
        (session_summary['session_length'] >= MIN_SESSION_LEN)
    ][out_session_col].tolist()
    df = df[df[out_session_col].isin(valid_sessions)].copy()
    df = df.drop(columns=['prev_ts','gap_s','new_session','sess_idx'], errors='ignore')
    return df, session_summary

def generate_prefix_target(df, session_col='session_id_real', item_col='item_id', max_prefix_len=MAX_PREFIX_LEN):
    """
    From a sessionized df ordered by timestamp, generate prefix->target pairs.
    Returns DataFrame with columns: dataset, user_id, session_id_real, prefix_len, prefix (space-separated), target
    """
    rows = []
    df = df.sort_values([session_col, 'timestamp'])
    grouped = df.groupby(session_col)
    for session_id, g in tqdm(grouped, desc="Generating prefix-target"):
        items = g[item_col].astype(str).tolist()
        if len(items) < 2: continue
        # optionally truncate to last (max_prefix_len + 1) items to limit explosion
        if len(items) > max_prefix_len + 1:
            items = items[-(max_prefix_len + 1):]
        for t in range(1, len(items)):
            prefix = items[:t]
            target = items[t]
            rows.append({
                'dataset': g['dataset'].iloc[0],
                'user_id': g['user_id'].iloc[0],
                session_col: session_id,
                'prefix_len': len(prefix),
                'prefix': " ".join(prefix),
                'target': str(target)
            })
        # flush periodically if very large
        if len(rows) >= 500_000:
            yield pd.DataFrame(rows)
            rows = []
    if rows:
        yield pd.DataFrame(rows)


In [5]:
# quick count per user (single-pass). If parquet fits in RAM you can do it with pandas.
import pandas as pd
from pathlib import Path
p = Path("../data/processed/amazon_books_2023_interactions.parquet")
df = pd.read_parquet(p, columns=["user_id"])   # only this col -> lower memory
vc = df["user_id"].value_counts()
vc.describe(percentiles=[0.5,0.75,0.9,0.99])
print("fraction users with count==1:", (vc==1).mean())


fraction users with count==1: 0.5772661819019989


In [6]:
p = Path("../data/processed/amazon_books_2023_interactions.parquet")
# load only user_id column (fast)
df_u = pd.read_parquet(p, columns=["user_id"])
vc = df_u["user_id"].value_counts()
for cut in [1,2,3,5,10]:
    kept = (vc >= cut).sum()
    print(f"users with count >= {cut}: {kept:,}  (fraction={kept/len(vc):.4f})")
# top-K example
for k in [100_000, 500_000, 1_000_000]:
    topk = vc.nlargest(k).index.size
    print(f"top {k} users: {topk}")


users with count >= 1: 9,614,012  (fraction=1.0000)
users with count >= 2: 4,064,168  (fraction=0.4227)
users with count >= 3: 2,315,212  (fraction=0.2408)
users with count >= 5: 1,102,772  (fraction=0.1147)
users with count >= 10: 380,903  (fraction=0.0396)
top 100000 users: 100000
top 500000 users: 500000
top 1000000 users: 1000000


In [7]:
import duckdb
duckdb.sql("""
COPY (
  SELECT *
  FROM read_parquet('../data/processed/amazon_books_2023_interactions.parquet')
  WHERE user_id IN (
    SELECT user_id FROM read_parquet('../data/processed/amazon_books_2023_interactions.parquet')
    GROUP BY user_id HAVING COUNT(*) >= 2
  )
) TO '../data/processed/amazon_active_count_ge2.parquet' (FORMAT PARQUET);
""")
print("Created amazon_active_count_ge2.parquet")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Created amazon_active_count_ge2.parquet


Step C — Build sliding windows from filtered file
Option C.1 — Single-process (simple, uses groupby)

In [8]:
WINDOW_SIZE = 20
STRIDE = 10
MIN_WINDOW_ITEMS = 2
OUT_DIR = Path("../data/processed")
in_path = OUT_DIR / "amazon_active_count_ge2.parquet"  # or amazon_top1M.parquet

df = pd.read_parquet(in_path)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df.sort_values(['user_id','timestamp'])

out_rows = []
out_parts = []
flush_every = 500000  # rows per part
part_idx = 0
count_rows = 0

for user, g in tqdm(df.groupby('user_id'), total=df['user_id'].nunique()):
    items = g['item_id'].astype(str).tolist()
    times = g['timestamp'].tolist()
    n = len(items)
    if n < MIN_WINDOW_ITEMS:
        continue
    start = 0
    win_idx = 0
    while start < n:
        end = min(start + WINDOW_SIZE, n)
        window_items = items[start:end]
        window_times = times[start:end]
        if len(window_items) >= MIN_WINDOW_ITEMS:
            sess_id = f"{user}__w{win_idx}_{start}"
            for it, ts in zip(window_items, window_times):
                out_rows.append({
                    "dataset": "amazon_books_2023",
                    "user_id": str(user),
                    "session_id_real": sess_id,
                    "item_id": str(it),
                    "timestamp": pd.to_datetime(ts)
                })
            win_idx += 1
        start += STRIDE
        if len(out_rows) >= flush_every:
            part = pd.DataFrame(out_rows)
            part_path = OUT_DIR / f"amazon_windows_sessions_part{part_idx:03d}.parquet"
            part.to_parquet(part_path, index=False)
            out_parts.append(part_path)
            count_rows += len(part)
            out_rows = []
            part_idx += 1

# final flush
if out_rows:
    part = pd.DataFrame(out_rows)
    part_path = OUT_DIR / f"amazon_windows_sessions_part{part_idx:03d}.parquet"
    part.to_parquet(part_path, index=False)
    out_parts.append(part_path)
    count_rows += len(part)

print("Wrote", len(out_parts), "parts; total rows:", count_rows)


100%|██████████| 4064168/4064168 [12:55<00:00, 5242.93it/s]


Wrote 55 parts; total rows: 27036156


In [9]:

DATA_DIR = Path("../data/processed")
parts = sorted(DATA_DIR.glob("amazon_windows_sessions_part*.parquet"))
print("Found", len(parts), "parts")

# quick totals
total_events = 0
for p in parts:
    # read only number of rows — with pandas you get memory for that part only
    dfp = pd.read_parquet(p, columns=["session_id_real"])
    total_events += len(dfp)
print("Total events across parts:", total_events)


Found 55 parts
Total events across parts: 27036156


In [11]:
import shutil

DATA_DIR = Path("../data/processed")
parts = sorted(DATA_DIR.glob("amazon_windows_sessions_part*.parquet"))
len(parts)
# create target folder
parts_dir = DATA_DIR / "amazon_windows_parts"
parts_dir.mkdir(exist_ok=True)

for p in parts:
    dest = parts_dir / p.name
    shutil.move(str(p), str(dest))

print("Moved", len(list(parts_dir.glob("amazon_windows_sessions_part*.parquet"))), "parts to", parts_dir)


Moved 55 parts to ..\data\processed\amazon_windows_parts


In [13]:
from pathlib import Path
import pandas as pd
from collections import Counter
import tqdm

parts_dir = Path("../data/processed/amazon_windows_parts")
parts = sorted(parts_dir.glob("amazon_windows_sessions_part*.parquet"))
print("Found", len(parts), "parts")

sess_counter = Counter()
total_events = 0

for p in tqdm.tqdm(parts, desc="Counting per-session lengths"):
    dfp = pd.read_parquet(p, columns=["session_id_real"])
    vc = dfp["session_id_real"].value_counts()
    total_events += len(dfp)
    # vc is a Series: index=session_id_real, value=count
    for sid, cnt in vc.items():
        sess_counter[sid] += int(cnt)

print("Total events counted across parts:", total_events)

# build DataFrame from Counter safely
sess_items = list(sess_counter.items())  # list of (session_id, length)
sess_df = pd.DataFrame(sess_items, columns=["session_id_real", "length"])
sess_df['length'] = sess_df['length'].astype(int)

print("Total sessions:", len(sess_df))
print(sess_df['length'].describe(percentiles=[.25,.5,.75,.9,.95,.99]))


Found 55 parts


Counting per-session lengths: 100%|██████████| 55/55 [00:11<00:00,  4.89it/s]


Total events counted across parts: 27036156
Total sessions: 4745988
count    4.745988e+06
mean     5.696634e+00
std      5.563629e+00
min      2.000000e+00
25%      2.000000e+00
50%      3.000000e+00
75%      6.000000e+00
90%      1.700000e+01
95%      2.000000e+01
99%      2.000000e+01
max      2.000000e+01
Name: length, dtype: float64


In [14]:
sess_df.to_csv(DATA_DIR / "amazon_windows_session_summary.csv", index=False)
print("Saved session summary to", DATA_DIR / "amazon_windows_session_summary.csv")


Saved session summary to ..\data\processed\amazon_windows_session_summary.csv


#### D — Generate prefix→target pairs for Amazon (stream chunked parts)
Read each window-session chunk and generate prefix-target pairs streaming to chunked parquet files.


In [15]:
parts_dir = Path("../data/processed/amazon_windows_parts")
pair_out_dir = Path("../data/processed/amazon_prefix_parts")
pair_out_dir.mkdir(exist_ok=True)

FLUSH = 200_000
pair_idx = 0
total_pairs = 0

def gen_pairs_from_df(df_part, session_col='session_id_real', item_col='item_id', max_prefix_len=50):
    df_part = df_part.sort_values([session_col, 'timestamp'])
    rows = []
    for session_id, g in df_part.groupby(session_col):
        items = g[item_col].astype(str).tolist()
        if len(items) < 2:
            continue
        if len(items) > max_prefix_len + 1:
            items = items[-(max_prefix_len + 1):]
        for t in range(1, len(items)):
            rows.append({
                'dataset': g['dataset'].iloc[0],
                'user_id': g['user_id'].iloc[0],
                session_col: session_id,
                'prefix_len': t,
                'prefix': " ".join(items[:t]),
                'target': str(items[t])
            })
            if len(rows) >= FLUSH:
                yield pd.DataFrame(rows)
                rows = []
    if rows:
        yield pd.DataFrame(rows)

# process each part file and write pair-parquet parts
for p in tqdm.tqdm(parts, desc="Generating prefix-target parts"):
    dfp = pd.read_parquet(p)
    for df_pairs in gen_pairs_from_df(dfp, max_prefix_len=50):
        outp = pair_out_dir / f"amazon_prefix_target_part{pair_idx:04d}.parquet"
        df_pairs.to_parquet(outp, index=False)
        total_pairs += len(df_pairs)
        pair_idx += 1
        print(f"Wrote {outp} (#pairs={len(df_pairs):,}); total_pairs={total_pairs:,}")

print("Done. Total prefix-target pairs:", total_pairs)


Generating prefix-target parts:   0%|          | 0/55 [00:00<?, ?it/s]

Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0000.parquet (#pairs=200,000); total_pairs=200,000
Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0001.parquet (#pairs=200,000); total_pairs=400,000


Generating prefix-target parts:   2%|▏         | 1/55 [00:20<18:06, 20.11s/it]

Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0002.parquet (#pairs=12,544); total_pairs=412,544
Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0003.parquet (#pairs=200,000); total_pairs=612,544
Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0004.parquet (#pairs=200,000); total_pairs=812,544


Generating prefix-target parts:   4%|▎         | 2/55 [00:40<17:42, 20.05s/it]

...

Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0158.parquet (#pairs=13,541); total_pairs=21,848,972
Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0159.parquet (#pairs=200,000); total_pairs=22,048,972
Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0160.parquet (#pairs=200,000); total_pairs=22,248,972


Generating prefix-target parts:  98%|█████████▊| 54/55 [18:55<00:21, 21.33s/it]

Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0161.parquet (#pairs=11,143); total_pairs=22,260,115


Generating prefix-target parts: 100%|██████████| 55/55 [18:57<00:00, 20.67s/it]

Wrote ..\data\processed\amazon_prefix_parts\amazon_prefix_target_part0162.parquet (#pairs=30,053); total_pairs=22,290,168
Done. Total prefix-target pairs: 22290168


In [16]:
# Cell: session-summary (fixed) + streaming prefix->target generation to a dedicated folder

DATA_DIR = Path("../data/processed")
parts_dir = DATA_DIR / "amazon_windows_parts"   # folder with amazon_windows_sessions_part*.parquet
pair_out_dir = DATA_DIR / "amazon_prefix_parts"
pair_out_dir.mkdir(exist_ok=True)

# --- 1) session-length aggregation across parts ---
parts = sorted(parts_dir.glob("amazon_windows_sessions_part*.parquet"))
print("Found parts:", len(parts))
sess_counter = Counter()
total_events = 0

for p in tqdm.tqdm(parts, desc="Counting per-session lengths"):
    dfp = pd.read_parquet(p, columns=["session_id_real"])
    vc = dfp["session_id_real"].value_counts()
    total_events += len(dfp)
    for sid, cnt in vc.items():
        sess_counter[sid] += int(cnt)

# convert Counter -> DataFrame
sess_items = list(sess_counter.items())
sess_df = pd.DataFrame(sess_items, columns=["session_id_real", "length"])
sess_df['length'] = sess_df['length'].astype(int)
print("Total events counted across parts:", total_events)
print("Total sessions:", len(sess_df))
print(sess_df['length'].describe(percentiles=[.25,.5,.75,.9,.95,.99]))

# Save session summary
sess_summary_path = DATA_DIR / "amazon_windows_session_summary_from_parts.csv"
sess_df.to_csv(sess_summary_path, index=False)
print("Saved session summary CSV to:", sess_summary_path)

# --- 2) Generate prefix->target parts streaming from window parts ---
FLUSH = 200_000   # number of pairs per output parquet file (tune if needed)
pair_idx = 0
total_pairs = 0

def gen_pairs_from_df(df_part, session_col='session_id_real', item_col='item_id', max_prefix_len=50):
    df_part = df_part.sort_values([session_col, 'timestamp'])
    rows = []
    for session_id, g in df_part.groupby(session_col):
        items = g[item_col].astype(str).tolist()
        if len(items) < 2:
            continue
        if len(items) > max_prefix_len + 1:
            items = items[-(max_prefix_len + 1):]
        for t in range(1, len(items)):
            rows.append({
                'dataset': g['dataset'].iloc[0],
                'user_id': g['user_id'].iloc[0],
                session_col: session_id,
                'prefix_len': t,
                'prefix': " ".join(items[:t]),
                'target': str(items[t])
            })
            if len(rows) >= FLUSH:
                yield pd.DataFrame(rows)
                rows = []
    if rows:
        yield pd.DataFrame(rows)

print("\nStart generating prefix->target parts (this can take time)...")
for p in tqdm.tqdm(parts, desc="Generating pairs from parts"):
    dfp = pd.read_parquet(p)
    for df_pairs in gen_pairs_from_df(dfp, max_prefix_len=50):
        outp = pair_out_dir / f"amazon_prefix_target_part{pair_idx:04d}.parquet"
        df_pairs.to_parquet(outp, index=False)
        total_pairs += len(df_pairs)
        pair_idx += 1
        print(f"Wrote {outp.name}  (#pairs={len(df_pairs):,})  total_pairs={total_pairs:,}")

print("\nDone generating prefix-target pairs.")
print("Total prefix-target parts written:", len(list(pair_out_dir.glob("amazon_prefix_target_part*.parquet"))))
print("Total pairs generated:", total_pairs)

# Save small manifest of parts for downstream notebooks
manifest = {
    "amazon_window_parts_dir": str(parts_dir),
    "amazon_window_parts_count": len(parts),
    "amazon_prefix_parts_dir": str(pair_out_dir),
    "amazon_prefix_parts_count": len(list(pair_out_dir.glob("amazon_prefix_target_part*.parquet"))),
    "total_pairs": total_pairs,
    "session_summary_csv": str(sess_summary_path)
}
with open(DATA_DIR / "amazon_parts_and_pairs_manifest.json", "w") as f:
    json.dump(manifest, f, indent=2)
print("Saved manifest:", DATA_DIR / "amazon_parts_and_pairs_manifest.json")


Found parts: 55


Counting per-session lengths: 100%|██████████| 55/55 [00:11<00:00,  4.75it/s]


Total events counted across parts: 27036156
Total sessions: 4745988
count    4.745988e+06
mean     5.696634e+00
std      5.563629e+00
min      2.000000e+00
25%      2.000000e+00
50%      3.000000e+00
75%      6.000000e+00
90%      1.700000e+01
95%      2.000000e+01
99%      2.000000e+01
max      2.000000e+01
Name: length, dtype: float64
Saved session summary CSV to: ..\data\processed\amazon_windows_session_summary_from_parts.csv

Start generating prefix->target parts (this can take time)...


Generating pairs from parts:   0%|          | 0/55 [00:00<?, ?it/s]

Wrote amazon_prefix_target_part0000.parquet  (#pairs=200,000)  total_pairs=200,000
Wrote amazon_prefix_target_part0001.parquet  (#pairs=200,000)  total_pairs=400,000


...

Wrote amazon_prefix_target_part0158.parquet  (#pairs=13,541)  total_pairs=21,848,972
Wrote amazon_prefix_target_part0159.parquet  (#pairs=200,000)  total_pairs=22,048,972
Wrote amazon_prefix_target_part0160.parquet  (#pairs=200,000)  total_pairs=22,248,972


Generating pairs from parts:  98%|█████████▊| 54/55 [18:44<00:22, 22.11s/it]

Wrote amazon_prefix_target_part0161.parquet  (#pairs=11,143)  total_pairs=22,260,115


Generating pairs from parts: 100%|██████████| 55/55 [18:46<00:00, 20.47s/it]

Wrote amazon_prefix_target_part0162.parquet  (#pairs=30,053)  total_pairs=22,290,168

Done generating prefix-target pairs.
Total prefix-target parts written: 163
Total pairs generated: 22290168
Saved manifest: ..\data\processed\amazon_parts_and_pairs_manifest.json


In [18]:
# Cell: quick sanity report (counts, top sessions, sample pair preview)

DATA_DIR = Path("../data/processed")
parts_dir = DATA_DIR / "amazon_windows_parts"
pair_out_dir = DATA_DIR / "amazon_prefix_parts"

# counts
window_parts = sorted(parts_dir.glob("amazon_windows_sessions_part*.parquet"))
pair_parts = sorted(pair_out_dir.glob("amazon_prefix_target_part*.parquet"))
print("Window parts:", len(window_parts))
print("Pair parts:", len(pair_parts))

# total events across window parts (fast)
total_window_events = sum(pd.read_parquet(p, columns=["session_id_real"]).shape[0] for p in window_parts)
print("Total window events (sum of parts):", total_window_events)

# total pairs across pair parts (fast)
total_pairs = sum(pd.read_parquet(p, columns=["prefix"]).shape[0] for p in pair_parts)
print("Total prefix-target pairs (sum of parts):", total_pairs)

# top sessions by length: read session summary csv if exists
sess_summary_csv = DATA_DIR / "amazon_windows_session_summary_from_parts.csv"
if sess_summary_csv.exists():
    sess_df = pd.read_csv(sess_summary_csv)
    top_sessions = sess_df.sort_values("length", ascending=False).head(10)
    print("\nTop 10 sessions by length (session_id_real, length):")
    print(top_sessions[["session_id_real","length"]].to_string(index=False))
else:
    print("Session summary CSV not found:", sess_summary_csv)

# sample one pair-part and show head
if pair_parts:
    sample_pair = pair_parts[0]
    df_sample = pd.read_parquet(sample_pair)
    print(f"\nSample pair file: {sample_pair.name}  shape={df_sample.shape}")
    display(df_sample.head(8))
else:
    print("No pair parts found in", pair_out_dir)


Window parts: 55
Pair parts: 163
Total window events (sum of parts): 27036156
Total prefix-target pairs (sum of parts): 22290168

Top 10 sessions by length (session_id_real, length):
                      session_id_real  length
AE4EQ5WN3XSND5SCGUA72VE4IUWQ__w12_120      20
AE4EQ5WN3XSND5SCGUA72VE4IUWQ__w13_130      20
AE4EQ5WN3XSND5SCGUA72VE4IUWQ__w14_140      20
AE4EQ5WN3XSND5SCGUA72VE4IUWQ__w15_150      20
AE4EQ5WN3XSND5SCGUA72VE4IUWQ__w16_160      20
   AE223J6LBWPONMIKQGHAI3X5GNOQ__w0_0      20
  AE223J6LBWPONMIKQGHAI3X5GNOQ__w1_10      20
   AFSZ5DKMKILWGCDM3MW3ZLFYPREQ__w0_0      20
   AFSZ5NHJ2Z4KHAQD5BXKIYZNJBBA__w0_0      20
   AFVEWKIUT76O6SWOWUJ224NHDWHQ__w0_0      20

Sample pair file: amazon_prefix_target_part0000.parquet  shape=(200000, 6)


,dataset,user_id,session_id_real,prefix_len,prefix,target
0,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,1,0061058386,0441008534
1,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,2,0061058386 0441008534,0441009239
2,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,3,0061058386 0441008534 0441009239,0375826688
3,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,4,0061058386 0441008534 0441009239 0375826688,B002KQ6BT6
4,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,5,0061058386 0441008534 0441009239 0375826688 B0...,0765362643
5,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,6,0061058386 0441008534 0441009239 0375826688 B0...,B002PEP4SC
6,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,7,0061058386 0441008534 0441009239 0375826688 B0...,B000R4LH3S
7,amazon_books_2023,AE22236AFRRSMQIKGG7TPTB75QEA,AE22236AFRRSMQIKGG7TPTB75QEA__w0_0,8,0061058386 0441008534 0441009239 0375826688 B0...,B0060QST0Q


#### E — Process MARS using 1-hour gap, then generate prefix-targets
This uses the sessionization-by-gap function and then generate prefix-target pairs (streamed).


In [22]:
# --- Cell: Process MARS with 1-hour gap and generate prefix->target pairs ---
import pandas as pd
from pathlib import Path
import tqdm
import json

DATA_DIR = Path("../data/processed")
mars_in = DATA_DIR / "mars_interactions.parquet"
assert mars_in.exists(), f"File not found: {mars_in}"

# Parameters
MARS_GAP_SECONDS = 60 * 60    # 1 hour
MIN_SESSION_LEN = 2
MAX_PREFIX_LEN = 50           # cap for prefixes (safe)
PAIR_FLUSH = 200_000          # flush pairs to disk in batches

# Load
df = pd.read_parquet(mars_in)
print("Loaded MARS interactions:", len(df))
print(df.dtypes)
# Ensure timestamp is datetime
df['timestamp'] = pd.to_datetime(df.get('timestamp', df.get('created_at')))

# Optional: convert explicit interactions to implicit-positive
# two common rules: rating >=3 OR watch_percentage >= 50. Apply whichever field exists.
if 'rating' in df.columns:
    # treat rating >= 3 as positive (change threshold if you prefer)
    df = df[df['rating'] >= 3].copy()
    print("Filtered by rating >= 3; remaining:", len(df))
elif 'watch_percentage' in df.columns:
    df = df[df['watch_percentage'] >= 50].copy()
    print("Filtered by watch_percentage >= 50; remaining:", len(df))
else:
    print("No rating/watch_percentage field found; keeping all interactions:", len(df))

# Sort
df = df.sort_values(['user_id', 'timestamp']).reset_index(drop=True)

# Sessionization by gap
df['prev_ts'] = df.groupby('user_id')['timestamp'].shift(1)
df['gap_s'] = (df['timestamp'] - df['prev_ts']).dt.total_seconds()
df['new_session'] = (df['gap_s'].isna()) | (df['gap_s'] > MARS_GAP_SECONDS)
df['sess_idx'] = df.groupby('user_id')['new_session'].cumsum().astype(int)
df['session_id_real'] = df['user_id'].astype(str) + "__s" + df['sess_idx'].astype(str)

# Optional: deduplicate consecutive identical items within same session
# This removes immediate repeats like [i,i,i] -> [i]
df = df.sort_values(['session_id_real', 'timestamp'])
df['prev_item'] = df.groupby('session_id_real')['item_id'].shift(1)
df = df[df['item_id'] != df['prev_item']].copy()
df = df.drop(columns=['prev_item'])

# Build session summary and filter by length >= MIN_SESSION_LEN
session_summary = (
    df.groupby('session_id_real')
      .agg(dataset=('user_id','first'), user_id=('user_id','first'),
           start_time=('timestamp','min'), end_time=('timestamp','max'),
           session_length=('item_id','size'))
      .reset_index()
)
valid_sessions = session_summary[session_summary['session_length'] >= MIN_SESSION_LEN]['session_id_real'].tolist()
print("Total sessions before filtering:", len(session_summary))
session_summary = session_summary[session_summary['session_length'] >= MIN_SESSION_LEN].reset_index(drop=True)
print("Total sessions after filtering (len >= 2):", len(session_summary))
print(session_summary['session_length'].describe())

# Keep only valid sessions in df
df = df[df['session_id_real'].isin(valid_sessions)].copy()
# drop helper cols
df = df.drop(columns=['prev_ts','gap_s','new_session','sess_idx'], errors='ignore')

# Save sessionized interactions and summary
mars_sessions_path = DATA_DIR / "mars_sessions.parquet"
mars_summary_path = DATA_DIR / "mars_session_summary.csv"
df.to_parquet(mars_sessions_path, index=False)
session_summary.to_csv(mars_summary_path, index=False)
print("Saved MARS sessions to:", mars_sessions_path)
print("Saved MARS session summary to:", mars_summary_path)

# Generate prefix->target pairs (streamed) and save in one parquet (small dataset ok in-memory)
pairs = []
for session_id, g in df.sort_values(['session_id_real','timestamp']).groupby('session_id_real'):
    items = g['item_id'].astype(str).tolist()
    # optional cap: keep last (MAX_PREFIX_LEN+1) items
    if len(items) > MAX_PREFIX_LEN + 1:
        items = items[-(MAX_PREFIX_LEN + 1):]
    for t in range(1, len(items)):
        prefix = items[:t]
        target = items[t]
        pairs.append({
            'dataset': 'mars',
            'user_id': g['user_id'].iloc[0],
            'session_id_real': session_id,
            'prefix_len': len(prefix),
            'prefix': " ".join(prefix),
            'target': str(target)
        })

mars_pairs_df = pd.DataFrame(pairs)
mars_pairs_path = DATA_DIR / "mars_prefix_target.parquet"
mars_pairs_df.to_parquet(mars_pairs_path, index=False)
print("Saved MARS prefix-target pairs to:", mars_pairs_path)
print("MARS pairs:", len(mars_pairs_df))

# Print quick diagnostics & sample
print("\n--- Diagnostics ---")
print("Interactions after filtering:", len(df))
print("Sessions (len>=2):", len(session_summary))
print("Pairs generated:", len(mars_pairs_df))
print(session_summary['session_length'].describe())

display(mars_pairs_df.head(8))


Loaded MARS interactions: 3659
dataset                     object
user_id                     object
session_id                  object
item_id                     object
timestamp           datetime64[ns]
interaction_type            object
dtype: object
No rating/watch_percentage field found; keeping all interactions: 3659
Total sessions before filtering: 1275
Total sessions after filtering (len >= 2): 549
count    549.000000
mean       5.335155
std        6.348793
min        2.000000
25%        2.000000
50%        3.000000
75%        5.000000
max       50.000000
Name: session_length, dtype: float64
Saved MARS sessions to: ..\data\processed\mars_sessions.parquet
Saved MARS session summary to: ..\data\processed\mars_session_summary.csv
Saved MARS prefix-target pairs to: ..\data\processed\mars_prefix_target.parquet
MARS pairs: 2380

--- Diagnostics ---
Interactions after filtering: 2929
Sessions (len>=2): 549
Pairs generated: 2380
count    549.000000
mean       5.335155
std        6.348

,dataset,user_id,session_id_real,prefix_len,prefix,target
0,mars,104074,104074__s2,1,52609,52616
1,mars,104074,104074__s2,2,52609 52616,52615
2,mars,104074,104074__s2,3,52609 52616 52615,52610
3,mars,104074,104074__s2,4,52609 52616 52615 52610,52614
4,mars,104074,104074__s2,5,52609 52616 52615 52610 52614,52618
5,mars,104074,104074__s2,6,52609 52616 52615 52610 52614 52618,52611
6,mars,104074,104074__s2,7,52609 52616 52615 52610 52614 52618 52611,52612
7,mars,104074,104074__s2,8,52609 52616 52615 52610 52614 52618 52611 52612,52617


#### F — Quick diagnostics to print & paste back for review

Please paste the outputs (numbers & small table) here after you run:
- # amazon parts written, total events written
- approximate # amazon sessions (from summaries) and session length stats
- total amazon prefix-target pairs generated
- mars sessions count, mars session length describe
- mars prefix-target pairs count and pairs.head()


In [23]:
# F: Quick diagnostics for all processed datasets

DATA_DIR = Path("../data/processed")

# Amazon windows & pair parts directories
amazon_windows_dir = DATA_DIR / "amazon_windows_parts"
amazon_pairs_dir = DATA_DIR / "amazon_prefix_parts"

# MARS paths
mars_sessions_path = DATA_DIR / "mars_sessions.parquet"
mars_pairs_path = DATA_DIR / "mars_prefix_target.parquet"
mars_summary_path = DATA_DIR / "mars_session_summary.csv"

# YOOCHOOSE path (already processed earlier)
yoo_path = DATA_DIR / "yoochoose_interactions.parquet"

# Gather file lists
amazon_window_parts = sorted(amazon_windows_dir.glob("amazon_windows_sessions_part*.parquet")) if amazon_windows_dir.exists() else []
amazon_pair_parts = sorted(amazon_pairs_dir.glob("amazon_prefix_target_part*.parquet")) if amazon_pairs_dir.exists() else []

print("=== Amazon (window parts) ===")
print("parts found:", len(amazon_window_parts))
total_window_events = sum(pd.read_parquet(p, columns=["session_id_real"]).shape[0] for p in amazon_window_parts) if amazon_window_parts else 0
print("total events across parts (sum):", total_window_events)

print("\n=== Amazon (pair parts) ===")
print("pair parts found:", len(amazon_pair_parts))
total_pairs = sum(pd.read_parquet(p, columns=["prefix"]).shape[0] for p in amazon_pair_parts) if amazon_pair_parts else 0
print("total prefix-target pairs (sum):", total_pairs)

print("\n=== MARS ===")
if mars_sessions_path.exists():
    df_mars = pd.read_parquet(mars_sessions_path)
    print("mars_sessions rows:", len(df_mars))
else:
    print("mars_sessions.parquet not found:", mars_sessions_path)
if mars_pairs_path.exists():
    df_mars_pairs = pd.read_parquet(mars_pairs_path)
    print("mars prefix-target pairs:", len(df_mars_pairs))
    display(df_mars_pairs.head(8))
else:
    print("mars_prefix_target.parquet not found:", mars_pairs_path)
if mars_summary_path.exists():
    print("mars session summary path exists:", mars_summary_path)
    print(pd.read_csv(mars_summary_path).describe(include='all'))
else:
    print("mars_session_summary.csv not found:", mars_summary_path)

print("\n=== YOOCHOOSE (basic check) ===")
if yoo_path.exists():
    df_yoo = pd.read_parquet(yoo_path, columns=["session_id","timestamp","item_id"])
    print("yoochoose events:", len(df_yoo))
    print("unique sessions:", df_yoo["session_id"].nunique())
    print("unique items:", df_yoo["item_id"].nunique())
else:
    print("yoochoose_interactions.parquet not found:", yoo_path)

# Print quick summary dict (for convenience)
summary = {
    "amazon_window_parts": len(amazon_window_parts),
    "amazon_window_events_sum": int(total_window_events),
    "amazon_pair_parts": len(amazon_pair_parts),
    "amazon_prefix_target_pairs_sum": int(total_pairs),
    "mars_sessions_rows": int(len(df_mars)) if 'df_mars' in locals() else None,
    "mars_pairs": int(len(df_mars_pairs)) if 'df_mars_pairs' in locals() else None,
    "yoo_events": int(len(df_yoo)) if 'df_yoo' in locals() else None
}

print("\nQuick summary (dict):")
print(json.dumps(summary, indent=2))



=== Amazon (window parts) ===
parts found: 55
total events across parts (sum): 27036156

=== Amazon (pair parts) ===
pair parts found: 163
total prefix-target pairs (sum): 22290168

=== MARS ===
mars_sessions rows: 2929
mars prefix-target pairs: 2380


,dataset,user_id,session_id_real,prefix_len,prefix,target
0,mars,104074,104074__s2,1,52609,52616
1,mars,104074,104074__s2,2,52609 52616,52615
2,mars,104074,104074__s2,3,52609 52616 52615,52610
3,mars,104074,104074__s2,4,52609 52616 52615 52610,52614
4,mars,104074,104074__s2,5,52609 52616 52615 52610 52614,52618
5,mars,104074,104074__s2,6,52609 52616 52615 52610 52614 52618,52611
6,mars,104074,104074__s2,7,52609 52616 52615 52610 52614 52618 52611,52612
7,mars,104074,104074__s2,8,52609 52616 52615 52610 52614 52618 52611 52612,52617


mars session summary path exists: ..\data\processed\mars_session_summary.csv
       session_id_real        dataset        user_id           start_time  \
count              549     549.000000     549.000000                  549   
unique             549            NaN            NaN                  549   
top         104074__s2            NaN            NaN  2019-07-22 15:27:08   
freq                 1            NaN            NaN                    1   
mean               NaN  400565.938069  400565.938069                  NaN   
std                NaN  147004.255186  147004.255186                  NaN   
min                NaN     672.000000     672.000000                  NaN   
25%                NaN  279346.000000  279346.000000                  NaN   
50%                NaN  404466.000000  404466.000000                  NaN   
75%                NaN  537638.000000  537638.000000                  NaN   
max                NaN  610262.000000  610262.000000                  NaN   

#### G — Save metadata (paths & counts) for downstream notebooks

Write a small JSON manifest so later notebooks can find files easily.


In [24]:
# G: Write a manifest file for downstream notebooks (paths + counts)

DATA_DIR = Path("../data/processed")
manifest_path = DATA_DIR / "sessionization_manifest.json"

# Build manifest content programmatically (read file lists and small counts)
amazon_windows_dir = DATA_DIR / "amazon_windows_parts"
amazon_pairs_dir = DATA_DIR / "amazon_prefix_parts"

def list_files_str(p, pattern):
    return [str(f) for f in sorted(p.glob(pattern))] if p.exists() else []

manifest = {
    "amazon_windows_parts_dir": str(amazon_windows_dir) if amazon_windows_dir.exists() else None,
    "amazon_windows_parts": list_files_str(amazon_windows_dir, "amazon_windows_sessions_part*.parquet"),
    "amazon_windows_events_sum": None,
    "amazon_prefix_parts_dir": str(amazon_pairs_dir) if amazon_pairs_dir.exists() else None,
    "amazon_prefix_parts": list_files_str(amazon_pairs_dir, "amazon_prefix_target_part*.parquet"),
    "amazon_prefix_pairs_sum": None,
    "mars_sessions": str(DATA_DIR / "mars_sessions.parquet") if (DATA_DIR / "mars_sessions.parquet").exists() else None,
    "mars_session_summary": str(DATA_DIR / "mars_session_summary.csv") if (DATA_DIR / "mars_session_summary.csv").exists() else None,
    "mars_prefix_target": str(DATA_DIR / "mars_prefix_target.parquet") if (DATA_DIR / "mars_prefix_target.parquet").exists() else None,
    "yoochoose_interactions": str(DATA_DIR / "yoochoose_interactions.parquet") if (DATA_DIR / "yoochoose_interactions.parquet").exists() else None
}

# fill in numeric sums if possible (cheap operations)
if manifest["amazon_windows_parts"]:
    manifest["amazon_windows_events_sum"] = sum(pd.read_parquet(p, columns=["session_id_real"]).shape[0] for p in amazon_windows_dir.glob("amazon_windows_sessions_part*.parquet"))
if manifest["amazon_prefix_parts"]:
    manifest["amazon_prefix_pairs_sum"] = sum(pd.read_parquet(p, columns=["prefix"]).shape[0] for p in amazon_pairs_dir.glob("amazon_prefix_target_part*.parquet"))

# write manifest
with open(manifest_path, "w") as f:
    json.dump(manifest, f, indent=2)

print("Wrote manifest to:", manifest_path)
print("Manifest summary:")
print(json.dumps({k: (len(v) if isinstance(v, list) else v) for k, v in manifest.items() if k.endswith('_parts') or k.endswith('_sum') or k.endswith('mars_prefix_target')}, indent=2))


Wrote manifest to: ..\data\processed\sessionization_manifest.json
Manifest summary:
{
  "amazon_windows_parts": 55,
  "amazon_windows_events_sum": 27036156,
  "amazon_prefix_parts": 163,
  "amazon_prefix_pairs_sum": 22290168,
  "mars_prefix_target": "..\\data\\processed\\mars_prefix_target.parquet"
}
